# ASync Example Bibliograpy Explorer

In [ ]:
import grpc
import json
import asyncio
from example_bib_app.grpc import example_bib_app_pb2_grpc, example_bib_app_pb2

from datetime import datetime

In [ ]:
# gRPC clients

grpc_server_host = 'localhost' # '127.0.0.1' 
grpc_server_port = 50051

sync_channel = grpc.aio.insecure_channel(f'{grpc_server_host}:{grpc_server_port}')
author_client = example_bib_app_pb2_grpc.AuthorControllerStub(sync_channel)
publisher_client = example_bib_app_pb2_grpc.PublisherControllerStub(sync_channel)

publ_cat_client = example_bib_app_pb2_grpc.PublicationCategoryControllerStub(sync_channel)

book_client = example_bib_app_pb2_grpc.BookControllerStub(sync_channel)

journal_client = example_bib_app_pb2_grpc.JournalControllerStub(sync_channel)

## Creating Authors

In [ ]:

for i in range(1000):
    author_response = await author_client.Create(example_bib_app_pb2.AuthorRequest(name_first='John', name_last=f'Doe{i}', birth_date=datetime.now().strftime('%Y-%m-%d')))
    author_response

In [ ]:
res = await author_client.List(example_bib_app_pb2.AuthorListRequest())
res.results
for author in res.results:
    print(author.name_first, author.name_last)
    last_author_id = author.author_id


In [ ]:
last_author_id

In [ ]:
publisher_response = await publisher_client.Create(example_bib_app_pb2.PublisherRequest(name='Hopper', website='https://hopper.com' ))
publisher_response

In [ ]:
# list publishers
res = await publisher_client.List(example_bib_app_pb2.PublisherListRequest())
res.results

## Using Filters

In [ ]:
filter_as_dict = {"name_last": "Doe"}
metadata = (("filters", (json.dumps(filter_as_dict))),)

res = await author_client.List(example_bib_app_pb2.AuthorListRequest(), metadata=metadata)
res.results


## Exploring Publisher

In [ ]:
# Create three Publisher

publisher_response = await publisher_client.Create(
    example_bib_app_pb2.PublisherRequest(
        name="O'Reilly",
        address="",
        city="",
        state_province="",
        country="",
        website="https://www.oreilly.com/",
    )
)
print("publisher create response:", publisher_response)

publisher_response = await publisher_client.Create(
    example_bib_app_pb2.PublisherRequest(
        name="Springer",
        address="",
        city="",
        state_province="",
        country="",
        website="https://www.springer.com/",
    )
)

print("publisher create response:", publisher_response)

publisher_response = await publisher_client.Create(
    example_bib_app_pb2.PublisherRequest(
        name="MIT Press",
        address="",
        city="",
        state_province="",
        country="",
        website="https://mitpress.mit.edu/",
    )
)

print("publisher create response:", publisher_response)

In [ ]:
# List Publishers

res = await publisher_client.List(example_bib_app_pb2.PublisherListRequest())
res.results

print("publisher list response:", res)

for publisher in res.results:
    print(publisher.name)
    last_publisher_id = publisher.publisher_id

In [ ]:
last_publisher_id

## Exploring Books

In [ ]:
# creating books
for i in range(1000):
    await book_client.Create(example_bib_app_pb2.BookRequest(title=f'book {i}', 
                                                            authors=[author_response.author_id],
                                                            isbn=f'isbn-{i}',
                                                            publisher=publisher_response.publisher_id,
                                                            publication_date=datetime.now().strftime('%Y-%m-%d')
                                                            ))

In [ ]:
# speed improved version, using gather

async def create_books(n):
    book_dict = {
        "title": f"book_gather {n}",
        "authors": [last_author_id],
        "isbn": f"isbn-{n}",
        "publisher": last_publisher_id,
        "publication_date": datetime.now().strftime("%Y-%m-%d"),
    }
    res = await book_client.Create(example_bib_app_pb2.BookRequest(**book_dict))
    return res

In [ ]:
await asyncio.gather(*(create_books(n) for n in range(1000)))

In [ ]:
# list books

res = await book_client.List(example_bib_app_pb2.BookListRequest())
res.results

In [ ]:
# stream books

async for book in  book_client.Stream(example_bib_app_pb2.BookStreamRequest()):
    print(book)


In [ ]:
# stream books
async for book in book_client.StreamBooksByIDList(example_bib_app_pb2.BookStreamBooksByIDListRequest(book_ids = ["d507ea75-a416-43c2-9de5-55a7e732b531", "0f82740f-f600-40a2-b7af-eb0af42ab975"])):
    print(book)

## Exploring Categories

In [ ]:
 # Create three PublicationCategories

publication_category_response = await publ_cat_client.Create(
    example_bib_app_pb2.PublicationCategoryRequest(
        name="Book",
    )
)
print("publication category create response:", publication_category_response)

publication_category_response = await publ_cat_client.Create(
    example_bib_app_pb2.PublicationCategoryRequest(
        name="Journal",
    )
)

print("publication category create response:", publication_category_response)

publication_category_response = await publ_cat_client.Create(
    example_bib_app_pb2.PublicationCategoryRequest(
        name="Magazine",
    )
)

print("publication category create response:", publication_category_response)


In [ ]:

# list all categories

res = await publ_cat_client.List(example_bib_app_pb2.PublicationCategoryListRequest())
res.results
for category in res.results:
    print(category.name)
    last_category_id = category.category_id

In [ ]:
last_category_id

## Exploring Journal Articles

In [ ]:
# creating Journal Articles
for i in range(10):
    await journal_client.Create(example_bib_app_pb2.JournalRequest(title=f'journal {i}', 
                                                            authors=[last_author_id],
                                                            publisher=last_publisher_id,
                                                            categories=[last_category_id],
                                                            #publication_date=datetime.now().strftime('%Y-%m-%d')
                                                            ))

In [ ]:
# list Journal Articles

res = await journal_client.List(example_bib_app_pb2.JournalListRequest())
res.results

## Closing channels

In [ ]:
sync_channel.close()